### Cuidados!
 
1. Whatsapp não gosta de nenhum tipo de automação
2. Isso pode dar merda, já to avisando
3. Isso não é o uso da API oficial do Whatsapp, o próprio whatsapp tem uma API oficial. Se o seu objetivo é fazer envio em massa ou criar aqueles robozinhos que respondem automaticamente no whatsapp, então use a API oficial
4. Meu objetivo é 100% educacional

### Dito isso, bora automatizar o envio de mensagens no Whatsapp

- Vamos usar o Selenium (vídeo da configuração na descrição)
- Temos 1 alternativa muito boa:
- Usar o https://web.whatsapp.com/send?phone=numero&text=texto (mais fácil, mais seguro, mas mais demorado)


In [1]:
import os
import time
import urllib
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

navegador = webdriver.Chrome(options=options)

def encontrar_elemento_botao(driver, metodo, seletor, tempo=20):
    return WebDriverWait(driver, tempo).until(EC.element_to_be_clickable((metodo, seletor)))


navegador.get("https://web.whatsapp.com")

## Importando a base de dados (site do whatsapp já carregado)

In [2]:
tabela_envio_mensagens = pd.read_excel("Envios.xlsx")

## Enviando mensagem para cada pessoa presente na base de dados

In [3]:
for linha in tabela_envio_mensagens.index:
    # armazenar informações
    nome = tabela_envio_mensagens.loc[linha, "nome"]
    mensagem = tabela_envio_mensagens.loc[linha, "mensagem"]
    arquivo_anexo = tabela_envio_mensagens.loc[linha, "arquivo"]
    telefone = tabela_envio_mensagens.loc[linha, "telefone"]

    texto = mensagem.replace("fulano", nome)
    texto = urllib.parse.quote(texto)

    # enviar mensagem para cada pessoa
    link = f"https://web.whatsapp.com/send?phone={telefone}&text={texto}"

    try:
        navegador.get(link)
        
        # aguardar até que a página seja carregada (elemento com id='side' apareça) ou timeout
        # while len(navegador.find_elements(By.ID, "side")) < 1: # -> Lista for vazia (elemento não existe ainda)
        #     time.sleep(1)
        WebDriverWait(navegador, 30).until(EC.presence_of_element_located((By.ID, "side")))
        time.sleep(3)

        # verificando se o número é inválido
        if len(navegador.find_elements(By.XPATH, '//*[@id="app"]/div/span[2]/div/span/div/div/div/div/div/div[1]')) < 1:
            # enviar a mensagem
            encontrar_elemento_botao(navegador, By.XPATH, '//*[@id="main"]/footer/div[1]/div/span/div/div[2]/div[2]/button/span').click()
            
            if arquivo_anexo != "N":
                caminho_completo_arquivo = os.path.abspath(f"arquivos/{arquivo_anexo}")
                encontrar_elemento_botao(navegador, By.XPATH, '//*[@id="main"]/footer/div[1]/div/span/div/div[1]/div/button/span').click()
                time.sleep(1)
                print(caminho_completo_arquivo)
                input_file = navegador.find_element(By.CSS_SELECTOR, 'input[type="file"]')
                input_file.send_keys(caminho_completo_arquivo)
                # encontrar_elemento_botao(navegador, By.XPATH, '//*[@id="app"]/div/span[5]/div/ul/div/div/div[1]/li').send_keys(caminho_completo_arquivo)
                time.sleep(10)
                encontrar_elemento_botao(navegador, By.XPATH, '//div[contains(@aria-label, "Enviar")]').click()

            time.sleep(5)
    except Exception as e:
        print(f'Erro encontrado: {e}')
    # finally:
    #     time.sleep(4)
    #     navegador.quit()


/Users/brunnomanduca/Documents/Repositórios/MyGithub/hashtag_python_impressionador/módulos/módulo29/Modulo29_aula32_Desafio_Automatizando_Envio_de_Mensagens_do_Whatsapp/arquivos/pythonimpressionador.png
